<a href="https://colab.research.google.com/github/diegonalvarez/machine-learning-group/blob/master/Predicciones_covid_Argentina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import operator
import datetime

from scipy.signal import find_peaks
import plotly.graph_objects as go

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures

In [0]:
confirmed_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
deaths_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'

cases_confirmed_ar = pd.read_csv(confirmed_url)
cases_deaths_ar = pd.read_csv(deaths_url)

In [3]:
cases_confirmed_ar.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,...,1,1,1,1,1,1,4,4,5,7,7,7,11,16,21,22,22,22,24,24,40,40,74,84,94,110,110,120,170,174,237,273,281,299,349,367,423,444,484,521
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,10,12,23,33,38,42,51,55,59,64,70,76,89,104,123,146,174,186,197,212,223,243,259,277,304,333,361,377,383,400,409,416
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,3,5,12,12,17,17,19,20,20,20,24,26,37,48,54,60,74,87,90,139,201,230,264,302,367,409,454,511,584,716,847,986,1171,1251,1320,1423,1468,1572,1666,1761
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,39,39,53,75,88,113,133,164,188,224,267,308,334,370,376,390,428,439,466,501,525,545,564,583,601
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,3,3,3,4,4,5,7,7,7,8,8,8,10,14,16,17,19,19,19


In [4]:
cases_deaths_ar.tail()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20
259,Saint Pierre and Miquelon,France,46.885200,-56.315900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
260,NaN,South Sudan,6.877000,31.307000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
261,NaN,Western Sahara,24.215500,-12.885800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
262,NaN,Sao Tome and Principe,0.186360,6.613081,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
263,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
cases_deaths_ar = cases_deaths_ar[(cases_deaths_ar['Country/Region']	== 'Argentina')]
cases_deaths_ar

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20
6,NaN,Argentina,-38.4161,-63.6167,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1,1,2,2,2,2,2,2,3,3,4,4,4,6,8,9,13,18,19,23,27,28,36,39,43,44,48,56,63,72,82


In [0]:
del cases_deaths_ar['Lat']
del cases_deaths_ar['Long']
del cases_deaths_ar['Province/State']

In [7]:
cases_confirmed_ar = cases_confirmed_ar[(cases_confirmed_ar['Country/Region']	== 'Argentina')]
cases_confirmed_ar

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20
6,NaN,Argentina,-38.4161,-63.6167,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,2,8,12,12,17,19,19,31,34,45,56,68,79,97,128,158,266,301,387,387,502,589,690,745,820,1054,1054,1133,1265,1451,1451,1554,1628,1715,1795,1975


In [0]:
del cases_confirmed_ar['Lat']
del cases_confirmed_ar['Long']
del cases_confirmed_ar['Province/State']

In [0]:
cases_confirmed_ar.rename(columns={'Country/Region':'Date'},inplace=True)
cases_confirmed_ar.rename(columns={'Argentina':'Total'},inplace=True)
cases_deaths_ar.rename(columns={'Country/Region':'Date'},inplace=True)
cases_deaths_ar.rename(columns={'Argentina':'Total'},inplace=True)

In [10]:
cases_confirmed_ar.head()

,Date,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,...,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20
6,Argentina,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,2,8,12,12,17,19,19,31,34,45,56,68,79,97,128,158,266,301,387,387,502,589,690,745,820,1054,1054,1133,1265,1451,1451,1554,1628,1715,1795,1975


In [0]:
cases_confirmed_ar = cases_confirmed_ar.T
cases_deaths_ar = cases_deaths_ar.T

In [12]:
cases_confirmed_ar

,6
Date,Argentina
1/22/20,0
1/23/20,0
1/24/20,0
1/25/20,0
...,...
4/6/20,1554
4/7/20,1628
4/8/20,1715
4/9/20,1795


cases_deaths_ar

In [0]:
cases_confirmed_ar.columns = cases_confirmed_ar.iloc[0]
cases_deaths_ar.columns = cases_deaths_ar.iloc[0]

In [14]:
cases_confirmed_ar

Date,Argentina
Date,Argentina
1/22/20,0
1/23/20,0
1/24/20,0
1/25/20,0
...,...
4/6/20,1554
4/7/20,1628
4/8/20,1715
4/9/20,1795


In [15]:
cases_deaths_ar.head()

Date,Argentina
Date,Argentina
1/22/20,0
1/23/20,0
1/24/20,0
1/25/20,0


In [0]:
cases_confirmed_ar = cases_confirmed_ar.iloc[1:]
cases_deaths_ar = cases_deaths_ar.iloc[1:]

In [17]:
cases_deaths_ar.head()

Date,Argentina
1/22/20,0
1/23/20,0
1/24/20,0
1/25/20,0
1/26/20,0


In [19]:
cases_confirmed_ar.rename(columns={'Argentina':'Total'}, inplace=True)
#cases_confirmed_ar.rename({'Country/Region':'Date'},axis=1)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [0]:
cases_deaths_ar = cases_deaths_ar.rename({'Country/Region':'Date'},axis=1)

In [21]:
cases_deaths_ar.head()

Date,Argentina
1/22/20,0
1/23/20,0
1/24/20,0
1/25/20,0
1/26/20,0


In [22]:
cases_confirmed_ar.head()

Date,Total
1/22/20,0
1/23/20,0
1/24/20,0
1/25/20,0
1/26/20,0


### Casos por día

In [0]:
#fig = go.Figure()

#grouped_df = cases_confirmed_ar.groupby(pd.Grouper(key='Date', freq='1d')).count().reset_index()
#fig.add_trace(go.Bar(x=grouped_df['fecha'], y=grouped_df['count']))
#fig = go.Figure(data=go.Scatter(x=grouped_df['Date'], y=cases_confirmed_ar.index)


#fig.update_layout(
 #   xaxis_tickformat = '%Y-%m-%d',
  #  xaxis_tickangle = -45,
  #  title_text="Cases by day"
#)
#fig.show()

In [0]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(y=cases_confirmed_ar.Total, x=cases_confirmed_ar.index ,mode='lines',name='Casos'))
fig.show()

### Regression model

In [0]:
proyeccion_dias = 10

In [38]:
cases_confirmed_ar.tail()

Date,Total,num_id
4/6/20,1554,76
4/7/20,1628,77
4/8/20,1715,78
4/9/20,1795,79
4/10/20,1975,80


In [52]:
from sklearn.model_selection import train_test_split

def polinomyal(x):
    polynomial_features= PolynomialFeatures(degree=3)
    x = x[:, np.newaxis] #crea acciones matriciales

    return polynomial_features.fit_transform(x)

def build_model(x, y):
    x = polinomyal(x)
    y = polinomyal(y)

    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=300)

    model = LinearRegression()
    return model.fit(X_train, y_train)

def get_predictions(model, x_predic): 
    x_predic = polinomyal(x_predic)
    y_predic = model.predict(x_predic)
    
    predictions = pd.DataFrame(columns=['Date', 'Total'])
    predictions['Date'] = x_predic[:, 1]
    predictions['Total'] = y_predic[:, 1]

    return predictions

import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(y=cases_confirmed_ar.Total, x=cases_confirmed_ar.index ,mode='lines',name='Casos'))
fig.show()

In [49]:
cases_confirmed_ar['num_id'] = np.arange(len(cases_confirmed_ar) + 1)[1:]
cases_confirmed_ar.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Date,Total,num_id
1/22/20,0,1
1/23/20,0,2
1/24/20,0,3
1/25/20,0,4
1/26/20,0,5


In [0]:
model = build_model(cases_confirmed_ar.num_id, cases_confirmed_ar.Total) 

In [51]:
predictions = get_predictions(model, np.arange(0, projeccion_dias))
print(predictions)

   Date       Total
0   0.0 -133.048063
1   1.0 -103.996073
2   2.0  -77.643969
3   3.0  -53.890414
4   4.0  -32.634070
5   5.0  -13.773599
6   6.0    2.792337
7   7.0   17.165074
8   8.0   29.445951
9   9.0   39.736306
